In [1]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

month = 1
year = 2024
path = Path("..") / "data" / "processed" / f"ts_data_{year}_{month:02}.parquet"

table = pq.read_table(path)
ts_data = table.to_pandas()
ts_data.head()

,pickup_hour,pickup_location_id,Zone,rides
0,2024-01-01 00:00:00,2,"Jamaica Bay, Queens",0
1,2024-01-01 01:00:00,2,"Jamaica Bay, Queens",0
2,2024-01-01 02:00:00,2,"Jamaica Bay, Queens",0
3,2024-01-01 03:00:00,2,"Jamaica Bay, Queens",0
4,2024-01-01 04:00:00,2,"Jamaica Bay, Queens",0


In [3]:
ts_data[ts_data["pickup_location_id"]==43].head(16)

,pickup_hour,pickup_location_id,Zone,rides
29760,2024-01-01 00:00:00,43,"Central Park, Manhattan",161
29761,2024-01-01 01:00:00,43,"Central Park, Manhattan",87
29762,2024-01-01 02:00:00,43,"Central Park, Manhattan",36
29763,2024-01-01 03:00:00,43,"Central Park, Manhattan",14
29764,2024-01-01 04:00:00,43,"Central Park, Manhattan",5
29765,2024-01-01 05:00:00,43,"Central Park, Manhattan",3
29766,2024-01-01 06:00:00,43,"Central Park, Manhattan",4
29767,2024-01-01 07:00:00,43,"Central Park, Manhattan",12
29768,2024-01-01 08:00:00,43,"Central Park, Manhattan",10
29769,2024-01-01 09:00:00,43,"Central Park, Manhattan",14


In [20]:
ts_data['Zone'] = ts_data['Zone'].astype(str)
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190464 entries, 0 to 190463
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         190464 non-null  datetime64[ns]
 1   pickup_location_id  190464 non-null  int32         
 2   Zone                190464 non-null  object        
 3   rides               190464 non-null  int64         
dtypes: datetime64[ns](1), int32(1), int64(1), object(1)
memory usage: 5.1+ MB


In [25]:
import pandas as pd
import numpy as np

def transform_time_series_to_tabular(df, location_id, feature_col="rides", window_size=12, step_size=1,location_name=None):
    """
    Transforms time series data for a given location ID into a tabular format.
    The first `window_size` rows are used as features, and the next row is the target.
    The process slides down by `step_size` rows at a time to create the next set of features and target.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing time series data.
        location_id (int): The location ID to filter the data for.
        feature_col (str): The column name containing the values to use as features and target (default is "rides").
        window_size (int): The number of rows to use as features (default is 12).
        step_size (int): The number of rows to slide the window by (default is 1).

    Returns:
        pd.DataFrame: A transformed DataFrame where the first `window_size` columns are features
                      and the last column is the target.
    """

    if location_name:
        location_data = df[df["Zone"] == location_name].reset_index(drop=True)
    else:
        location_data = df[df["pickup_location_id"] == location_id].reset_index(drop=True)

    # Extract the feature column as a NumPy array
    values = location_data[feature_col].values

    # Ensure there are enough rows to create at least one window
    if len(values) <= window_size:
        raise ValueError("Not enough data to create even one window of features and target.")

    # Create the tabular data using a sliding window approach
    rows = []
    for i in range(0, len(values) - window_size, step_size):
        # The first `window_size` values are features, and the next value is the target
        features = values[i:i + window_size] 
        target = values[i + window_size] 
        rows.append(np.append(features, target))

    # Convert the list of rows into a DataFrame
    column_names = [f"feature_{i+1}" for i in range(window_size)] + ["target"]
    transformed_df = pd.DataFrame(rows, columns=column_names)

    return transformed_df

In [26]:
features_targets = transform_time_series_to_tabular(ts_data, 43, "rides", 24, 1)

In [27]:
features_targets

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,target
0,161,87,36,14,5,3,4,12,10,14,...,123,82,50,39,39,36,24,13,5,3
1,87,36,14,5,3,4,12,10,14,28,...,82,50,39,39,36,24,13,5,3,0
2,36,14,5,3,4,12,10,14,28,55,...,50,39,39,36,24,13,5,3,0,0
3,14,5,3,4,12,10,14,28,55,48,...,39,39,36,24,13,5,3,0,0,0
4,5,3,4,12,10,14,28,55,48,73,...,39,36,24,13,5,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,92,93,55,38,12,3,0,0,1,1,...,89,89,103,119,73,126,93,107,118,79
716,93,55,38,12,3,0,0,1,1,8,...,89,103,119,73,126,93,107,118,79,51
717,55,38,12,3,0,0,1,1,8,11,...,103,119,73,126,93,107,118,79,51,53
718,38,12,3,0,0,1,1,8,11,47,...,119,73,126,93,107,118,79,51,53,39
